# -- XGB Model --

In [1]:
#libs
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import numpy as np

In [2]:
#dataframe
df = pd.read_csv('df.csv')
df.head()

,Margem EBITDA,Divida liquida/EBTIDA,Indice de liquidez,Ciclo Financeiro,Conversao Ebitda em FCO,Tempo de atuação em anos,Auditorias,PEFIN,Garantia,Seguros,Serasa Score,Target
0,8.44,-0.94,1.94,101,31.81,4,6,0,1,2,900,1
1,3.70,2.23,0.62,99,-10.16,3,0,1,0,0,50,0
2,8.00,1.72,0.59,118,73.67,6,2,2,1,0,200,1
3,3.35,2.19,0.51,51,81.34,2,3,3,2,0,400,1
4,8.61,-0.17,2.40,21,69.11,4,2,0,2,3,550,1


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48721 entries, 0 to 48720
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Margem EBITDA             48721 non-null  float64
 1   Divida liquida/EBTIDA     48721 non-null  float64
 2   Indice de liquidez        48721 non-null  float64
 3   Ciclo Financeiro          48721 non-null  int64  
 4   Conversao Ebitda em FCO   48721 non-null  float64
 5   Tempo de atuação em anos  48721 non-null  int64  
 6   Auditorias                48721 non-null  int64  
 7   PEFIN                     48721 non-null  int64  
 8   Garantia                  48721 non-null  int64  
 9   Seguros                   48721 non-null  int64  
 10  Serasa Score              48721 non-null  int64  
 11  Target                    48721 non-null  int64  
dtypes: float64(4), int64(8)
memory usage: 4.5 MB


## Treinamento do Modelo XGB com Grid Search CV

In [4]:
# 1. Separar features (X) e target (y)
X = df.drop(['Target'], axis=1) # Suas features originais
y = df['Target']             

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

print(f"Dimensões do X_train: {X_train.shape}")
print(f"Dimensões do X_test: {X_test.shape}")

Dimensões do X_train: (34104, 11)
Dimensões do X_test: (14617, 11)


In [5]:
# Definir a grade de parâmetros
param_grid_xgb = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7],
    'subsample': [0.7, 0.8, 0.9],
    'colsample_bytree': [0.7, 0.8, 0.9]
}

# Iniciar o modelo
xgb_base = xgb.XGBClassifier(
    objective='binary:logistic',
    eval_metric='logloss',
    random_state=42,
    tree_method='hist'
)

# Configurar o GridSearchCV
grid_search_xgb = GridSearchCV(
    estimator=xgb_base,
    param_grid=param_grid_xgb,
    scoring='roc_auc', 
    cv=5,              # 5-fold cross-validation
    n_jobs=-1,         # Usar todos os núcleos da CPU disponíveis
    verbose=1          # Para ver o progresso
)

print("\nIniciando GridSearchCV para XGBoost...")
grid_search_xgb.fit(X_train, y_train)

print(f"\nMelhores parâmetros para XGBoost: {grid_search_xgb.best_params_}")
print(f"Melhor ROC AUC com XGBoost (CV): {grid_search_xgb.best_score_:.4f}")

# Avaliar o melhor estimador no conjunto de teste
best_xgb_model = grid_search_xgb.best_estimator_
y_pred_best_xgb = best_xgb_model.predict(X_test)
y_pred_proba_best_xgb = best_xgb_model.predict_proba(X_test)[:, 1]

print("\n--- Avaliação do XGBoost Otimizado no Conjunto de Teste ---")
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_best_xgb))
print("\nClassification Report:")
print(classification_report(y_test, y_pred_best_xgb))
print(f"\nROC AUC Score para XGBoost Otimizado: {roc_auc_score(y_test, y_pred_proba_best_xgb):.4f}")


Iniciando GridSearchCV para XGBoost...
Fitting 5 folds for each of 243 candidates, totalling 1215 fits

Melhores parâmetros para XGBoost: {'colsample_bytree': 0.7, 'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 300, 'subsample': 0.7}
Melhor ROC AUC com XGBoost (CV): 0.9997

--- Avaliação do XGBoost Otimizado no Conjunto de Teste ---
Confusion Matrix:
[[4777   16]
 [ 108 9716]]

Classification Report:
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      4793
           1       1.00      0.99      0.99      9824

    accuracy                           0.99     14617
   macro avg       0.99      0.99      0.99     14617
weighted avg       0.99      0.99      0.99     14617


ROC AUC Score para XGBoost Otimizado: 0.9997


In [6]:
importances_xgb = best_xgb_model.get_booster().get_score(importance_type='gain')

# Criar um DataFrame para facilitar a ordenação
feature_importance_df_xgb = pd.DataFrame({
    'Feature': list(importances_xgb.keys()),
    'Importance': list(importances_xgb.values())
})

In [7]:
# Ordenar as features por importância
feature_importance_df_xgb = feature_importance_df_xgb.sort_values(by='Importance', ascending=False)

print("\n--- Feature Importances do XGBoost (Top 10) ---")
print(feature_importance_df_xgb.head(10))


--- Feature Importances do XGBoost (Top 10) ---
                     Feature  Importance
1      Divida liquida/EBTIDA  543.977539
0              Margem EBITDA  133.009079
2         Indice de liquidez  131.934509
6                 Auditorias   35.530491
4    Conversao Ebitda em FCO   29.103680
8                   Garantia   23.847261
10              Serasa Score   22.895164
9                    Seguros   20.362888
5   Tempo de atuação em anos   11.318794
7                      PEFIN    9.544507


## Teste de utilização do modelo.
* Desempacotar o .pkl e passar um novo dicionário de dados para o modelo dar uma reposta baseada no aprendizado.

In [14]:
import joblib

# --- Carregar o Modelo Salvo ---
try:
    modelo = joblib.load('best_xgboost_credit_model.pkl')
    print("Modelo XGBoost salvo com sucesso!")
    print(f"Tipo do objeto carregado: {type(modelo)}") # Adicione esta linha para verificar
except FileNotFoundError:
    print("Erro: O arquivo 'best_xgboost_credit_model.pkl' não foi encontrado.")
    exit()

Modelo XGBoost salvo com sucesso!
Tipo do objeto carregado: <class 'xgboost.sklearn.XGBClassifier'>


In [15]:
indicadores = ['Margem EBITDA', 'Divida liquida/EBTIDA',  'Indice de liquidez',
                  'Ciclo Financeiro', 'Conversao Ebitda em FCO', 'Tempo de atuação em anos', 
                 'Auditorias', 'PEFIN',  'Garantia', 'Seguros',  'Serasa Score', ]

# --- Nova amostra de dados ---

amostra = [
    5.15,       # Margem EBITDA
    -2.23,      # Divida liquida/EBTIDA 
    1.9,        # Indice de liquidez
    30,         # Ciclo Financeiro
    0.2,        # Conversao Ebitda em FCO
    3,          # Tempo de atuação em anos
    1,          # Auditorias 
    1,          # PEFIN 
    1,          # Garantia 
    1,          # Seguros 
    850,        # Serasa Score
]

In [16]:
if len(amostra) != len(indicadores):
    raise ValueError(f"O número de valores na lista ({len(amostra)}) não corresponde ao número de features ({len(indicadores)})")
amostra_df = pd.DataFrame([amostra], columns=indicadores)
print("\nDados da nova empresa para previsão:")
amostra_df


Dados da nova empresa para previsão:


,Margem EBITDA,Divida liquida/EBTIDA,Indice de liquidez,Ciclo Financeiro,Conversao Ebitda em FCO,Tempo de atuação em anos,Auditorias,PEFIN,Garantia,Seguros,Serasa Score
0,5.15,-2.23,1.9,30,0.2,3,1,1,1,1,850


In [17]:
# Prever a classe 0 ou 1 (reprovado ou aprovado)
previsao = modelo.predict(amostra_df)
print(f"\nResultado da Previsão da Amostra (0 = Negar, 1 = Conceder): {previsao[0]}")


Resultado da Previsão da Amostra (0 = Negar, 1 = Conceder): 1


In [22]:
proba = modelo.predict_proba(amostra_df)
print(f"Probabilidade de Negar Crédito (Classe 0): {proba[0][0]*100:.2f}%")
print(f"Probabilidade de Conceder Crédito (Classe 1): {proba[0][1]*100:.2f}%")

Probabilidade de Negar Crédito (Classe 0): 5.24%
Probabilidade de Conceder Crédito (Classe 1): 94.76%
